In [4]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd


%load_ext autoreload
%autoreload 1
%aimport bb_behavior
%aimport bb_behavior.plot
%aimport bb_behavior.tracking
%aimport bb_behavior.tracking.pipeline

import bb_behavior
import bb_behavior.plot
import bb_behavior.tracking
import bb_behavior.tracking.pipeline

from bb_behavior.tracking.pipeline import get_default_pipeline
from tqdm import tqdm_notebook # progress bar
import math
from bb_tracking.data.constants import DETKEY
#from bb_tracking.tracking import score_id_sim_v
from bb_tracking.tracking import distance_orientations_v, distance_positions_v

from bb_behavior.tracking.pipeline import detect_markers_in_video
from bb_behavior.tracking.pipeline import track_detections_dataframe
from bb_behavior.tracking.pipeline import display_tracking_results

Using Theano backend.


In [5]:
import time
import os

def filename_to_datestring(filname):
    """
    filename can be path
    """
    return os.path.split(filname)[-1].split('.')[0].split('_')[1]


def string_to_timestamp(datestring):
    """ 
    params
        string: format 2018-08-19-01-08-13
    output
        unix timestamp (float)
    """
    
    return time.mktime(time.strptime(datestring, "%Y-%m-%d-%H-%M-%S"))

def timestamp_to_string(timestamp):
    return time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(timestamp))    

In [6]:
paths = glob.glob(os.path.join('../videos/videos_tags/', '*.h264'))
datestring = filename_to_datestring(paths[0])
timestamp = string_to_timestamp(datestring)
string =  timestamp_to_string(timestamp)

print(paths[0])
print(datestring)
print(timestamp)
print(string)

../videos/videos_tags/e00_2018-08-19-01-08-13.h264
2018-08-19-01-08-13
1534633693.0
2018-08-19-01-08-13


In [ ]:
time.mktime(time.strptime(datestring, "%Y-%m-%d-%H-%M-%S"))

In [ ]:
time.localtime(timestamp)

In [ ]:
# hyperparameters

tag_pixel_diameter = 50
n_frames = None
fps = 10.0
progress = "tqdm_notebook"
tracker = "../tracker.det_score_fun.frag_score_fun.dill"
confidence_filter_detections = 0.08
confidence_filter_tracks = 0.2
coordinate_scale = 1.0
localizer_threshold="0.55"

#1944

default_pipeline = get_default_pipeline(localizer_threshold=localizer_threshold)

track_dfs = []

counter = 0
for path in glob.glob(os.path.join('../videos/videos_tags/', '*.h264')):  

    print(counter, ": ", path)
    
    try:

        # get the detections
        frame_info, detections = detect_markers_in_video(path,
                                                decoder_pipeline=default_pipeline,
                                                tag_pixel_diameter=tag_pixel_diameter,
                                                n_frames=n_frames,
                                                fps=fps,
                                                progress=progress)

        print(len(detections))
        # get the tracks                                           
        tracks = track_detections_dataframe(detections,
                                            tracker=tracker,
                                            confidence_filter_detections=confidence_filter_detections,
                                            confidence_filter_tracks=confidence_filter_tracks,
                                            coordinate_scale=coordinate_scale)
        # get date from filename
        date_string = os.path.split(path)[-1].split('.')[0].split('_')[1]
        timestamp = time.mktime(datetime.datetime.strptime(date_string, "%Y-%m-%d-%H-%M-%S").timetuple())

        #  transform tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
        grouped = tracks.groupby('track_id',  as_index=False)['bee_id','xpos','ypos','timestamp']
        df = grouped.aggregate(lambda x: list(x))
        df = df.drop('track_id', 1)

        df['timestamp_video'] = timestamp
        df = df.rename(columns={'timestamp': 'timestamps'})

        # convert bee_id list to single value
        df['bee_id'] = df['bee_id'].apply(lambda x: x[0])

        # only appen if note detections?
        track_dfs.append(df)

        if counter == 4: 
            break
        counter += 1
    
    except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
        print(err)
    except Exception as err:
        print(err)
        raise
    
# bind dataframes together
tracks = pd.concat(track_dfs,ignore_index=True)

In [ ]:
# merge tracks of same bee where start and end timestamps are close together
# assume there can not be overlapping tracks
# 1. sort: bee_id, start_time

# calculate start time of track by adding timestamp of track (seconds since start of video)
# to timestamp of video (date)
tracks['start_time'] = tracks['timestamp_video'] + tracks['timestamps'].apply(lambda x: x[0])
tracks['end_time'] = tracks['timestamp_video'] + tracks['timestamps'].apply(lambda x: x[-1])

In [ ]:
tracks3 = tracks.copy()

In [ ]:
tracks['timestamps'].apply(lambda x: x[-1]) - tracks['timestamps'].apply(lambda x: x[0])

In [ ]:
tracks['end_time'] - tracks['start_time']

In [ ]:
tracks

In [ ]:
tracks = tracks3.copy()
tracks = tracks.sort_values(['bee_id', 'start_time'])

index = 0
tracks2 = tracks.copy()
while(True):
    row = tracks.iloc[index]
    next_row = tracks.iloc[index+1]
    
    # if the tracks are from same bee and the start time of next row is closer then 10s -> merge the rows
    # merge rows means, next_row is deleted
    if row['bee_id'] == next_row['bee_id'] and next_row['start_time'] - row['end_time'] < 10:
        print(next_row['start_time'] - row['end_time'], row['bee_id'], next_row['bee_id'])

        # update the timestamps of nextrow
        t = next_row['start_time'] - row['start_time']
        timestamps = list(np.array(next_row['timestamps']) + t)
        
        tracks.at[row.name,'xpos'] = row['xpos']+next_row['xpos']
        tracks.at[row.name,'ypos'] = row['ypos']+next_row['ypos']
        tracks.at[row.name,'timestamps'] = row['timestamps']+timestamps
        tracks.at[row.name,'end_time'] = next_row['end_time']
        
        tracks.drop(tracks.index[index+1], inplace=True)

    else:
        index += 1
    
    if index == len(tracks) - 1:
        break

In [ ]:
tracks.iloc[0]["timestamps"]

In [ ]:
tracks = tracks3.copy()
tracks = tracks.sort_values(['bee_id', 'start_time'])
for index, rows in tracks.iterrows():
    row = tracks.iloc[index]
    next_row = tracks.iloc[index+1]
    print(next_row['start_time'] - row['end_time'], row['bee_id'], next_row['bee_id'])

In [ ]:
tracks3

In [ ]:
tracks2

In [ ]:
tracks2

In [ ]:
list(np.array([1,2,3,4,5]))

In [ ]:
tracks.iloc[0]

In [ ]:
# hyperparameters

#e00_2018-08-19-02-20-55

tag_pixel_diameter = 50
n_frames = None
fps = 10.0
progress = "tqdm_notebook"
tracker = "../tracker.det_score_fun.frag_score_fun.dill"
confidence_filter_detections = 0.08 # 0.08
confidence_filter_tracks = 0.2 #.2
coordinate_scale = 1.0
localizer_threshold="0.50" # 50

#1944


default_pipeline = get_default_pipeline(localizer_threshold=localizer_threshold)

track_dfs = []
counter = 0
for path in ['../videos/videos_tags/e00_2018-08-19-01-08-13.h264']:  

    print(counter, ": ", path)
    
    try:

        # get the detections
        frame_info, detections = detect_markers_in_video(path,
                                                decoder_pipeline=default_pipeline,
                                                tag_pixel_diameter=tag_pixel_diameter,
                                                n_frames=n_frames,
                                                fps=fps,
                                                progress=progress)

        print(len(detections))
        # get the tracks                                           
        tracks = track_detections_dataframe(detections,
                                            tracker=tracker,
                                            confidence_filter_detections=confidence_filter_detections,
                                            confidence_filter_tracks=confidence_filter_tracks,
                                            coordinate_scale=coordinate_scale)
        
        display_tracking_results(path, frame_info, detections, tracks)

    
    except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
        print(err)
    except Exception as err:
        print(err)
        raise